### MRO

> MRO — это порядок, в котором Python ищет методы и атрибуты при их вызове. Он особенно важен в контексте множественного наследования, где один класс может наследовать поведение от нескольких других классов. MRO позволяет Python решать, какой метод или атрибут использовать, если они определены в нескольких родительских классах. 

> В Python для вычисления MRO используется алгоритм C3 Linearization (C3-линеаризация).  
>>Два основных правила линеаризации:  
>>__-дети идут раньше родителей;__  
>>__-родители идут в порядке перечисления.__

In [ ]:
def show(_mro):
    c = [f'({v.__name__})' for v in _mro]
    return ' -> '.join(c)

In [ ]:
class A1:
    ...

class B1(A1):
    ...

print(show(B1.__mro__))

(B) -> (A) -> (object)


In [ ]:
class A2:
    ...

class B2(A2):
    ...

class C2(B2):
    ...

print(show(C2.__mro__))

(C2) -> (B2) -> (A2) -> (object)


In [ ]:
# Diamond problem
class A3:
    ...

class B3(A3):
    ...

class C3(A3):
    ...

class D3(B3, C3):
    ...

class E3(C3,  B3):
    ...


print(show(D3.__mro__))
print(show(E3.__mro__))

(D3) -> (B3) -> (C3) -> (A3) -> (object)
(E3) -> (C3) -> (B3) -> (A3) -> (object)


In [ ]:
class A4:
    ...

class B4(A4):
    ...

class C4(A4):
    ...

class D4(B4, C4):
    ...

class E4(C4,  B4):
    ...

class F5(D4, E4):
    ...

print(show(F5.__mro__))

TypeError: Cannot create a consistent method resolution
order (MRO) for bases B4, C4

> Одним из часто используемых механизмов, связанных с MRO, является функция `super()`. Она позволяет обращаться к методам родительских классов, что особенно полезно при переопределении методов в дочерних классах.

In [ ]:
class A6:
    def who_am_i(self):
        print("I am A")
 
class B6(A6):
    def who_am_i(self):
        super().who_am_i()
        print("I am B")
 
class C6(A6):
    def who_am_i(self):
        super().who_am_i()
        print("I am C")
 
class D6(B6, C6):
    def who_am_i(self):
        super().who_am_i()
        print("I am D")
 
d = D6()
d.who_am_i()
print(show(D6.__mro__))

I am A
I am C
I am B
I am D
(D6) -> (B6) -> (C6) -> (A6) -> (object)


> при НЕРОМБОВИДНОЙ схеме, наследование производится классически (сначала в глубину, затем слева направо) 

In [ ]:
class A7: pass
class B7: pass
class C7(A7): pass
class D7(B7): pass
class E7(C7, D7): pass
print(show(E7.__mro__))

(E7) -> (C7) -> (A7) -> (D7) -> (B7) -> (object)


### Генераторы

> Генератор — это функция, которая возвращает итератор (объект-генератор).  
> Функции-генераторы помогают получать значения по запросу.  
> Объект-генератор можно обойти только один раз.  
> Функции-генераторы экономят память.  
> После возврата всех элементов(прохождения всего цикла), генератор вызовет исключение StopIteration.  
> Инструкция yield возвращает следующее значение и приостанавливает работу функции-генератора.  
> yield можно использовать несколько раз в коде функции-генератора.  
> Помимо yield генератор может содержать и return. Встретив return генератор выбрасывает исключение  StopIteration, а возвращенное значение записывается в объект StopIteration в атрибут value (по умолчанию None)!  

In [ ]:
# пример функции-генератора
def gen(n):
    for i in range(1, n):
        yield i*2

a = gen(10)            # a - объект-генератор !
print(next(a))
print(next(a))

2
4


In [10]:
# пример генераторного выражения
a = (x*2 for x in range(1, 10))
print(next(a))
print(next(a))

2
4


> У объекта-генератора есть три метода:  
> - .close() - остановка выполнения генератора;  
> - .throw() - поднятие исключения (генератор должен быть инициализирован вызовом next или send(None));  
> - .send() - отправить значение генератору. Чтобы начать передачу значений в генератор его нужно сначала инициализировать с помощью вызова `next()` или отправив в генератор значение None.

In [11]:
# пример с исключением
def gen_exept(n):
    for i in range(1, 10):
        yield i*2

b = gen_exept(10)
for val in b:
    if val == 6:
        b.throw(Exception('Val is 6!'))
    print(val)

2
4


Exception: Val is 6!

### Дескрипторы

> Дескрипторы - объекты, которые реализуют методы `__get__`, `__set__`, `__delete__`, позволяют переопределять стандартное поведение при работе с атрибутами объекта, добавляют логику при чтении, записи или удалении значения атрибута.  
> Существует необязательный метод `__set_name__`, который позволяет дескриптору узнавать имя атрибута, к которому он присвоен в классе.  

> Дескрипторы применяются при валидации данных, вычислениях значений, управлении доступом к атрибутам, реализации сложной логики.  

> Дескрипторы создаются в виде независимых классов, имеют собственное состояние, и присваиваются атрибутам классов в точности как функции методов.  

In [ ]:
class Descriptor:
    """docstring"""                             # Строка документации
    def __get__(self, instance, owner): ...     # Возвращает значение атрибута
    def __set__(self, instance, value): ...     # Ничего не возвращает (None)
    def __delete__ (self, instance): ...        # Ничего не возвращает (None)

> *Всем трем методам дескриптора, передаются экземпляр класса дескриптора **(self)** и экземпляр клиентского класса **(instance)**, к которому присоединен экземпляр дескриптора. Метод доступа `__get__` также принимает аргумент **(owner)**, указывающий класс, к которому присоединен экземпляр дескриптора.*

In [41]:
class DescriptorA:
    def __get__(self, inst, ow):
        return f'{self}, {inst}, {ow}'
    
class DAMain:
    ds = DescriptorA()


dam = DAMain()
print(dam.ds)

<__main__.DescriptorA object at 0x0000014A94F7BB60>, <__main__.DAMain object at 0x0000014A94F7B140>, <class '__main__.DAMain'>
Hello!


> Если метод `__set__` не переопределен в дескрипторе то он работает по умолчанию.

In [ ]:
dam.ds = 'Hello!'       # вызов dam.ds.__set__()
print(dam.ds)

In [42]:
# Пример
class Name:
    "descriptor docs"
    def __get__(self, instance, owner):
        return f'Hello, {instance._name}'
    
    def __set__(self, instance, value):
        instance._name = value
    
    def __delete__(self, instance):
        del instance._name

        
class Person:
    def __init__(self, name):
        self._name = name
    name = Name()

p = Person('Maks')
print(p.name)
p.name = 'John'
print(p.name)

Hello, Maks
Hello, John


> Связать конкретный дескриптор с конкретным атрибутом:

In [44]:
class DescriptorB:
    def __set_name__(self, owner, attr_name: str) -> None:
        self._attr_name = attr_name

    def __get__(self, instance, owner=None):
        return instance.__dict__[self._attr_name]

    def __set__(self, instance, value) -> None:
        instance.__dict__[self._attr_name] = value

        
class DBMain:
    attr_1 = DescriptorB()

    
instance_object = DBMain()
instance_object.attr_1 = 100
instance_object.attr_1  # -> 100

100

> Если объект дескриптора определяет методы `__get__`, `__set__` или `__delete__`, то он считается data дескриптором;  
> Если объект дескриптора определяет только метод `__get__`, то он является non-data дескриптором.  
> Data и non-data дескрипторы отличаются по порядку поиска атрибутов и предоставлению доступа к ним.

### Аннотация типов данных

In [46]:
# аннотация переменной
a: int
a = 4
print(a)

b: float = 4.57
print(b)

4
4.57


In [47]:
# функции
def funcA(a: int, b: str) -> float:
    ...

print(funcA.__annotations__)

{'a': <class 'int'>, 'b': <class 'str'>, 'return': <class 'float'>}


In [ ]:
# списка
l: list[float] = [2.56, 6.1, 23.9]

In [ ]:
# кортежа, для каждого элемента обозначить тип !
t: tuple[int, str, bool] = (3, 'three', True)
# кортеж с произвольным числом элементов
t2: tuple[int, ...]

In [ ]:
# словаря
d: dict[int, float] = {1: 3.145}

In [ ]:
# множества
st: set[int] = {2, 6, 1, 9, 0}

> Для версий языка Python ниже 3.9 используют типы List, Tuple, Dict, Set из модуля typing

#### Модуль typing и типы Union, Optional, Any, Callable

In [51]:
from typing import Union, Optional, Any, Callable

> Union позволяет комбинировать несколько разных типов в один, образуя составной тип.

In [ ]:
a: Union[int, float]
# или
b: int | float

> Optional позволяет указать один какой-либо тип данных и еще автоматически добавляется тип None.

In [ ]:
s: Optional[str]       # эквивалентно s: Union[str, None]

> Any означает буквально любой тип данных.

In [ ]:
c: Any

> Callable позволяет аннотировать вызываемые объекты. Часто это обычные функции, которые передаются как параметры.  
> В общем случае тип Callable описывается по синтаксису: `Callable[[TypeArg1, TypeArg2, …], ReturnType]`  

In [53]:
def calculate(i: int) -> float:
    return 0.5 * i

def main_func(call_func: Callable[[int], Union[int, float]], 
              n: Union[int, float]) -> Union[int, float]:
    return call_func(n)

print(main_func(calculate, 43))

21.5


In [ ]:
# комбинирование
co: list[Union[int, float, bool]]
co2: Optional[list[Union[int, float]], tuple[int, ...]]

#### Аннотация с помощью Type и TypeVar

> С помощью класса TypeVar можно описать некий общий тип.  

In [66]:
from typing import Type, TypeVar, Generic

In [64]:
T = TypeVar('T')

def mirror(value: list[Type[T]]) -> Type[T]:
    return value[0]

print(mirror([1, 2, 3]))
print(mirror(['1', '2', '3']))

1
1


In [ ]:
class MainT:
    var = 'Main'

class ChildT(MainT):
    var = 'Child'

class Fictive:
    var = 'Fictive'

# здесь объявление универсального типа с именем T
# bound указывает что он должен быть или классом MainT или любым его дочерним классом
T = TypeVar('T', bound=MainT)

def factory(obj: Type[T]) -> str:
    return obj.var

print(factory(MainT))
print(factory(ChildT))
print(factory(Fictive))  # здесь подсказка анализатору, код выполнится 

Main
Child
Fictive


In [ ]:
# обобщенный тип
T1 = TypeVar('T1')
T2 = TypeVar('T2')

class Pair(Generic[T1, T2]):
    def __init__(self, first: T1, second: T2):
        self.first = first
        self.second = second

    def get_first(self) -> T1:
        return self.first

    def get_second(self) -> T2:
        return self.second
    
pair = Pair(1, "one")
print(pair.get_first())     # выведет 1
print(pair.get_second())    # выведет one

1
one


#### typing.Annotated

> typing.Annotated — это аннотация, позволяющая расширить тип дополнительными метаданными.

> Первым аргументом в Annotated всегда указывается валидный тип (например: int), все последующие аргументы являются метаданными.  
> Метаданными могут являться любые объекты python. Они могут использоваться либо для статического анализа, либо во время выполнения.  
> Если инструмент проверки типизации сталкивается с подсказкой типа Annotated[T, x] и не имеет специальной логики для метаданных x, то он должен игнорировать его и просто рассматривать как тип T.

In [70]:
from typing import Annotated
x: Annotated[int, 'need integer'] = 45

> Для получения аннотаций типов из объектов (функций, методов, классов или модулей) с метаданными можно использовать функцию `get_type_hints` из модуля typing с указанием аргумента `include_extras=True`.  Метаданные хранятся в атрибуте `__metadata__`.  

In [75]:
from typing import get_type_hints

def func_annot(x: int, y: str, z: Annotated[float, 'need float']) -> bool:
    ...
    return True

print(get_type_hints(func_annot))
print(get_type_hints(func_annot, include_extras=True))                      # включение Annotated
print(get_type_hints(func_annot, include_extras=True)['z'].__metadata__)    # метаданные параметра z

{'x': <class 'int'>, 'y': <class 'str'>, 'z': <class 'float'>, 'return': <class 'bool'>}
{'x': <class 'int'>, 'y': <class 'str'>, 'z': typing.Annotated[float, 'need float'], 'return': <class 'bool'>}
('need float',)
